In [1]:
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
)
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

model_path = './models/llama-2-7b-chat.Q4_K_M.gguf'
# model_path = './models/mistral-7b-instruct-v0.2.Q4_K_M.gguf'

# 1. Set up local LLM

In [2]:
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    # model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    # model_path='./models/llama-2-13b-chat.Q5_0.gguf',
    model_path=model_path,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,    
    completion_to_prompt=completion_to_prompt,        
    verbose=True,
)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [ ]:
response_iter = llm.stream_complete("Can you write me a poem about fast cars?")
for response in response_iter:
    print(response.delta, end="", flush=True)

# 2. Query engine

In [3]:
from llama_index import set_global_tokenizer
from transformers import AutoTokenizer

set_global_tokenizer(
    AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf").encode
)

# use Huggingface embeddings
from llama_index.embeddings import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
text_embedding = embed_model.get_text_embedding("hello world")
print(len(text_embedding))

# create a service context
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
)

# load documents
documents = SimpleDirectoryReader(
    input_files=["./docs/eBook-How-to-Build-a-Career-in-AI.pdf", "./docs/recipes.pdf", "./docs/annualreport.pdf"]
).load_data()

# create vector store index
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)

# set up query engine
query_engine = index.as_query_engine(streaming=True)

384


# 3. Test query engine

In [ ]:
# helper functions to print out the response
def query(query_str):
    streaming_response = query_engine.query(query_str)
    streaming_response.print_response_stream()

In [ ]:
# answer retrieved from eBook-How-to-Build-a-Career-in-AI.pdf
query("how do I get started on a personal project in AI?")
# query("How do I build a portfolio of AI projects?")
# query("Summarize the book in 500 words.")

In [ ]:
# answer retrieved from annualreport.pdf
query("what was the FY2022 return on equity?")

In [ ]:
# answer retrieved from recipes.pdf
query("How to make Pineapple Chicken?")

# 4. Window-sentence retrieval setup (Advanced retrieval)

In [21]:
import os
from llama_index import ServiceContext, VectorStoreIndex, StorageContext
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index import load_index_from_storage


def build_sentence_window_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=3,
    save_dir="sentence_index",
):
    # create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentence_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
    )
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex.from_documents(
            documents, service_context=sentence_context
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
        )

    return sentence_index


def get_sentence_window_query_engine(
    sentence_index, similarity_top_k=6, rerank_top_n=2
):
    # define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
    )
    return sentence_window_engine

In [22]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

index = build_sentence_window_index(
    [document],
    llm=llm,
    save_dir="./sentence_index",
)

In [23]:
query_engine = get_sentence_window_query_engine(index, similarity_top_k=6)

In [24]:
response = query_engine.query("How do I build a portfolio of AI projects?")
print(response.response)

Llama.generate: prefix-match hit


  Great! Based on the provided context, here's how you can build a portfolio of AI projects:
1. Start small: As mentioned in the context, don't worry about starting too small. Begin with simple projects that demonstrate your understanding of AI concepts and techniques. This will help you gain confidence and build momentum for more complex projects.
2. Focus on practical applications: While it's important to have a solid theoretical foundation in AI, it's equally crucial to show how your skills can be applied in real-world scenarios. Choose projects that demonstrate practical applications of AI, such as image classification, natural language processing, or predictive modeling.
3. Show progression: As you build your portfolio, aim to show a clear progression in complexity and scope of projects. This will help potential employers or clients understand how you've grown and developed as an AI practitioner over time.
4. Communicate your thinking: As mentioned in the context, communication is

In [25]:
response = query_engine.query("what was the FY2022 return on equity?")
print(response.response)

Llama.generate: prefix-match hit


  Based on the provided context information, the FY2022 return on equity (ROE) for Macquarie Group Limited is 18.7%. This can be found on page 3 of the Annual Report under the heading "Remuneration Committee Letter" where it is stated that "ROE of 18.7% is up compared to FY2021's 14.3%."


In [26]:
response = query_engine.query("How to make Pineapple Chicken?")
print(response.response)

Llama.generate: prefix-match hit


  Sure, I'd be happy to help you with that! Based on the context information provided, here is the step-by-step guide on how to make Pineapple Chicken:
Step 1: Heat oil in a large skillet over medium-high heat. Add all ingredients EXCEPT pineapple and chicken. Cook and stir until heated through, about 5-6 minutes.
Step 2: Add pineapple and chicken to the skillet. Cook for another 2 minutes.
That's it! You now have a delicious Pineapple Chicken dish that you can serve over instant brown rice or whole wheat pasta.
I hope this helps, and please let me know if you have any questions or need further clarification.
